# Integral PI (with @reduction)

## 1.1 Initialize PyCOMPSs

In [1]:
import pycompss.interactive as ipycompss

In [2]:
import os
if 'BINDER_SERVICE_HOST' in os.environ:
    ipycompss.start(graph=True,                # trace=True
                   project_xml='../xml/project.xml',
                    resources_xml='../xml/resources.xml')
else:
    ipycompss.start(graph=True, monitor=1000, debug=True)  # trace=True

********************************************************
**************** PyCOMPSs Interactive ******************
********************************************************
*          .-~~-.--.           ______         ______   *
*         :         )         |____  \       |____  \  *
*   .~ ~ -.\       /.- ~~ .      __) |          __) |  *
*   >       `.   .'       <     |__  |         |__  |  *
*  (         .- -.         )   ____) |   _    ____) |  *
*   `- -.-~  `- -'  ~-.- -'   |______/  |_|  |______/  *
*     (        :        )           _ _ .-:            *
*      ~--.    :    .--~        .-~  .-~  }            *
*          ~-.-^-.-~ \_      .~  .-~   .~              *
*                   \ \ '     \ '_ _ -~                *
*                    \`.\`.    //                      *
*           . - ~ ~-.__\`.\`-.//                       *
*       .-~   . - ~  }~ ~ ~-.~-.                       *
*     .' .-~      .-~       :/~-.~-./:                 *
*    /_~_ _ . - ~              

 

* - Log path : /home/javier/.COMPSs/Interactive_09/
* - PyCOMPSs Runtime started... Have fun!              *
********************************************************


* - Log path : /home/user/.COMPSs/Interactive_22/
* - PyCOMPSs Runtime started... Have fun!              *
********************************************************


## 1.2 Required imports

In [3]:
from pycompss.api.api import compss_wait_on
from pycompss.api.task import task
from pycompss.api.reduction import reduction
from pycompss.api.parameter import *

## 2 Tasks Declaration

In [4]:
import numpy as np

In [5]:
@task(returns=float)
def calculate_area(i, num_steps, number_of_batches, step_size):
    partial_area_sum = 0
    for i in range(i, num_steps, number_of_batches):
        x = (i+0.5) * step_size
        partial_area_sum += 4 / (1 + x**2)
    return partial_area_sum

![title](pi_resources/pi_integral.png)

In [6]:
@reduction(chunk_size="2")
@task(returns=float, batches_partial_areas=COLLECTION_IN)
def sum_reduction(batches_partial_areas):
    total_area = 0
    for partial_area in batches_partial_areas:
        total_area += partial_area
    return total_area

Run the algorithm

In [7]:
num_steps = 100000
number_of_batches = 10

In [8]:
step_size = 1 / num_steps

In [9]:
batches_partial_areas = []
for i in range(number_of_batches):
    partial_area = calculate_area(i, num_steps, number_of_batches, step_size)
    batches_partial_areas.append(partial_area)

    
total_area = sum_reduction(batches_partial_areas)

Task definition detected.
Found task: calculate_area
Task definition detected.
Found task: sum_reduction


Wait for all tasks to finish and gather the result

In [10]:
total_area = compss_wait_on(total_area)

Calculate PI

In [11]:
pi = step_size * total_area

In [12]:
print('PI:', pi, 'Error:', abs(np.pi-pi))

PI: 3.141592653598127 Error: 8.333778112046275e-12


In [13]:
ipycompss.stop()

********************************************************
***************** STOPPING PyCOMPSs ********************
********************************************************
Checking if any issue happened.
         have not been brought to the master.
********************************************************
